In [1]:
import pandas as pd
import numpy as np
import xarray as xr

import os
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from pyaldata import *

In [3]:
# Load data
data_dir = ""
fname = os.path.join(data_dir, "Chewie_CO_CS_2016-10-14.mat")

In [4]:
# load TrialData .mat file into a DataFrame
df = mat2dataframe(fname, shift_idx_fields=True)

In [6]:
# Preprocessing

# combine time bins into longer ones
td = combine_time_bins(df, 3)
print(df.head().bin_size)
print()
print(td.head().bin_size)

0    0.01
1    0.01
2    0.01
3    0.01
4    0.01
Name: bin_size, dtype: float64

0    0.03
1    0.03
2    0.03
3    0.03
4    0.03
Name: bin_size, dtype: float64


In [7]:
# Keep only successful trials
print("all trials:", td.shape[0])

td = select_trials(td, "result == 'R'")

print("successful trials:", td.shape[0])

all trials: 770
successful trials: 740


In [8]:
# Remove low-firing neurons
td = remove_low_firing_neurons(td, "M1_spikes",  5)
td = remove_low_firing_neurons(td, "PMd_spikes", 5)

print("original: ", df.M1_spikes[0].shape)
print("removed:  ", td.M1_spikes[0].shape)

original:  (359, 88)
removed:   (119, 63)


c:\users\jon\documents\biomedical engineering - ic\4. letnik\final year project\python\pyaldata\pyaldata\tools.py:979: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")


In [9]:
# Get the signal from idx_go_cue
print('Orig:', td.M1_spikes[0].shape)
td = restrict_to_interval(td, start_point_name='idx_go_cue', end_point_name='idx_trial_end')
print('New:', td.M1_spikes[0].shape)

Orig: (119, 63)
New: (40, 63)


In [10]:
# Smooting the signal

td = smooth_signals(td, ["M1_spikes", "PMd_spikes"], std=0.05)



In [11]:
# Combine M1 and PMd
td = merge_signals(td, ["M1_spikes", "PMd_spikes"], "both_spikes")

In [12]:
N = td.shape[0]


for i in range(N):
    
    
    # get position
    pos = td.pos[i]

    
    # get velocities for this trial
    vel = td.vel[i]
   
    
    # get M1 spikes
    M1 = td.M1_spikes[i]
    
    # get PMd_spikes
    PMd = td.PMd_spikes[i]
    
    # get both
    both_spikes = td.both_spikes[i]
    
    
    if (i == 0):
        grouped_pos = pos
        grouped_vel = vel

        grouped_M1 = M1
        grouped_PMd = PMd
        grouped_both_spikes = both_spikes
    else:
        grouped_pos = np.append(grouped_pos, pos, axis=0)
        grouped_vel = np.append(grouped_vel, vel, axis=0)
        grouped_M1 = np.append(grouped_M1, M1, axis=0)
        grouped_PMd = np.append(grouped_PMd, PMd, axis=0)
        grouped_both_spikes = np.append(grouped_both_spikes, both_spikes, axis=0);
 
print(grouped_pos.shape)    
print(grouped_vel.shape)
print(grouped_M1.shape)
print(grouped_PMd.shape)
print(grouped_both_spikes.shape)
   
    

(30068, 2)
(30068, 2)
(30068, 63)
(30068, 120)
(30068, 183)


In [13]:
# Save the data in right format
import pickle

data_folder='../preprocessed_data/' #FOLDER YOU WANT TO SAVE THE DATA TO

with open(data_folder+'all_data.pickle','wb') as f:
    pickle.dump([grouped_M1, grouped_PMd, grouped_both_spikes, grouped_pos, grouped_vel],f)